In [ ]:
import tensorflow.compat.v1 as tf
print(tf.__version__)

In [ ]:


import pandas as pd

import pickle

import tensorflow as tf


dataset = pd.read_csv("transliteration.txt",delimiter = "\t",header=None,encoding='utf-8',na_filter = False)


X = dataset.iloc[:,0]
y = dataset.iloc[:,-1]


import Data_preprocessing



source_int_text, target_int_text, source_vocab_to_int, target_vocab_to_int,source_int_to_vocab,target_int_to_vocab = Data_preprocessing.preprocess(X,y)


import Layers
import Model_Inputs


def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  target_sequence_length,
                  max_target_word_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):


    enc_outputs, enc_states = Layers.encoding_layer(input_data, 
                                             rnn_size, 
                                             num_layers, 
                                             keep_prob, 
                                             source_vocab_size, 
                                             enc_embedding_size)
    
    dec_input = Model_Inputs.process_decoder_input(target_data, 
                                      target_vocab_to_int, 
                                      batch_size)
    
    train_output, infer_output = Layers.decoding_layer(dec_input,
                                               enc_states, 
                                               target_sequence_length, 
                                               max_target_word_length,
                                               rnn_size,
                                              num_layers,
                                              target_vocab_to_int,
                                              target_vocab_size,
                                              batch_size,
                                              keep_prob,
                                              dec_embedding_size)
    
    return train_output, infer_output




display_step = 200

epochs = 60
batch_size = 30

rnn_size = 64
num_layers = 2

encoding_embedding_size = 50
decoding_embedding_size = 50

learning_rate = 0.001
keep_probability = 0.5


save_path = 'checkpoints/dev'



train_graph = tf.Graph()
with train_graph.as_default():

    input_data, targets, target_sequence_length, max_target_sequence_length = Model_Inputs.enc_dec_model_inputs()
    lr, keep_prob = Model_Inputs.hyperparam_inputs()
    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)
    
    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')


    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        
        optimizer = tf.train.AdamOptimizer(lr)

        
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
        



train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]


import Batch_Metrics

(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(Batch_Metrics.get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  


with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                Batch_Metrics.get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 keep_prob: keep_probability})

        
            if batch_i % display_step == 0 and batch_i > 0:
                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})

                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = Batch_Metrics.get_accuracy(target_batch, batch_train_logits)
                valid_acc = Batch_Metrics.get_accuracy(valid_targets_batch, batch_valid_logits)
                print('Epoch {:>3} Batch {:>3}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i+1, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))


    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and saved')
    
  
def save_params(params):
    with open('params.p', 'wb') as out_file:
        pickle.dump(params, out_file)

save_params(save_path)


In [ ]:
import tensorflow as tf
import pickle
import Data_preprocessing

def load_params():
    with open('params.p', mode='rb') as in_file:
        return pickle.load(in_file)

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = Data_preprocessing.load_preprocess()

load_path = load_params()



batch_size = 30

def word_to_seq(word, vocab_to_int):
    results = []
    for word in list(word):
        if word in vocab_to_int:
            results.append(vocab_to_int[word])
        else:
            results.append(vocab_to_int['<UNK>'])
            
    return results

print("\n Enter word to be transliterated:")
transliterate_word = input().lower()



transliterate_word = word_to_seq(transliterate_word, source_vocab_to_int)

loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as sess:
        
    loader = tf.train.import_meta_graph(load_path + '.meta')
    
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    transliterate_logits = sess.run(logits, {input_data: [transliterate_word]*batch_size,
                                         target_sequence_length: [len(transliterate_word)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in transliterate_word]))
print('  English Word: {}'.format([source_int_to_vocab[i] for i in transliterate_word]))

print('\nPrediction')
print('  Word Id:      {}'.format([i for i in transliterate_logits]))

output = ""
for i in transliterate_logits:
        if target_int_to_vocab[i]!= '<EOS>':
                output = output + target_int_to_vocab[i]
print('  Hindi Word:      {}'.format(output))